In [1]:
# import all libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import re
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import r2_score
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score






import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import scale
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import LeaveOneOut



import warnings # supress warnings
warnings.filterwarnings('ignore')


pd.set_option('display.max_columns', None)

df=pd.read_csv('qb_data_new.csv')
df=df.drop(columns=['Unnamed: 0'])

In [41]:
df2=df[df['passer_player_name'] == 'B.Mayfield']

In [42]:
df2

,passer_player_name,week,season,posteam,defteam,complete_pass,incomplete_pass,interception,qb_hit,sack,pass_touchdown,passing_yards,pass_attempts,def_complete_pass_allowed,def_avg_ints,def_avg_qb_hit,def_avg_sack,def_avg_td_allowed,def_avg_yards_allowed,2023_qb_completion_avg,2023_qb_yards_avg,2023_qb_attempts_avg,2023_qb_int_avg,age,dvoa,anya
0,B.Mayfield,1,2019,CLE,TEN,25.0,10.0,3.0,6.0,5.0,1.0,285.0,35.0,18.250000,0.250000,3.375000,2.125000,1.000000,206.000000,22.363636,235.272727,33.909091,0.636364,28,-5.9,6.1
1,B.Mayfield,10,2023,TB,TEN,18.0,10.0,1.0,5.0,3.0,2.0,278.0,28.0,18.250000,0.250000,3.375000,2.125000,1.000000,206.000000,22.363636,235.272727,33.909091,0.636364,28,-5.9,6.1
2,B.Mayfield,13,2020,CLE,TEN,25.0,8.0,0.0,0.0,0.0,4.0,334.0,33.0,18.250000,0.250000,3.375000,2.125000,1.000000,206.000000,22.363636,235.272727,33.909091,0.636364,28,-5.9,6.1
3,B.Mayfield,1,2020,CLE,BAL,21.0,17.0,1.0,6.0,2.0,1.0,189.0,38.0,21.500000,1.000000,5.875000,3.625000,0.750000,188.250000,22.363636,235.272727,33.909091,0.636364,28,-5.9,6.1
4,B.Mayfield,4,2019,CLE,BAL,20.0,9.0,1.0,1.0,1.0,1.0,342.0,29.0,21.500000,1.000000,5.875000,3.625000,0.750000,188.250000,22.363636,235.272727,33.909091,0.636364,28,-5.9,6.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,B.Mayfield,12,2019,CLE,MIA,24.0,9.0,1.0,3.0,1.0,3.0,327.0,33.0,22.875000,0.500000,6.625000,2.750000,1.750000,241.000000,22.363636,235.272727,33.909091,0.636364,28,-5.9,6.1
81,B.Mayfield,12,2020,CLE,JAX,19.0,10.0,0.0,1.0,2.0,2.0,258.0,29.0,18.750000,0.875000,4.875000,1.625000,1.375000,213.875000,22.363636,235.272727,33.909091,0.636364,28,-5.9,6.1
82,B.Mayfield,14,2018,CLE,CAR,18.0,4.0,0.0,1.0,1.0,1.0,238.0,22.0,19.857143,0.142857,4.571429,1.428571,1.571429,201.428571,22.363636,235.272727,33.909091,0.636364,28,-5.9,6.1
83,B.Mayfield,15,2022,LA,GB,12.0,8.0,1.0,8.0,5.0,1.0,111.0,20.0,19.500000,0.375000,5.250000,1.875000,1.000000,209.500000,22.363636,235.272727,33.909091,0.636364,28,-5.9,6.1


In [74]:
columns_to_keep = ['passing_yards', 'dvoa', 'anya']

In [43]:
df2=df2[columns_to_keep]

In [44]:
X = df2.drop('passing_yards', axis=1)
y = df2['passing_yards']

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) 
X_test_scaled = scaler.transform(X_test)

In [17]:
lm = LinearRegression()
lm.fit(X_train_scaled, y_train)

LinearRegression()

In [18]:
y_pred = lm.predict(X_test_scaled)

In [19]:
# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae}')

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error (MSE): {mse}')

# Calculate Root Mean Squared Error (RMSE)
rmse = mse**0.5
print(f'Root Mean Squared Error (RMSE): {rmse}')

Mean Absolute Error (MAE): 39.99416022631862
Mean Squared Error (MSE): 2616.7475490777783
Root Mean Squared Error (RMSE): 51.154154758707314


In [21]:
r_squared = r2_score(y_test, y_pred)
print(r_squared)


-0.04820204706468778


In [23]:
mean_y_baseline = np.mean(y_test)
y_baseline = np.full_like(y_test, fill_value=mean_y_baseline)
rmse_baseline = np.sqrt(mean_squared_error(y_test, y_baseline))
print(f'RMSE for Baseline (Predicting Mean of Y): {rmse_baseline}')

mse = mean_absolute_error(y_test, y_baseline)
print(f'MAE): {mse}')

RMSE for Baseline (Predicting Mean of Y): 49.964139389300954
MAE): 40.45674740484429


XGBOOST

In [49]:
import xgboost as xgb


dtrain_reg = xgb.DMatrix(X_train_scaled, y_train,)
dtest_reg = xgb.DMatrix(X_test_scaled, y_test)

In [50]:
# Set parameters
params = {'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'max_depth': 3}

# Train the model
model = xgb.train(params, dtrain_reg, num_boost_round=100)

# Make predictions
predictions = model.predict(dtest_reg)

In [51]:
# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error (MAE): {mae}')

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error (MSE): {mse}')

# Calculate Root Mean Squared Error (RMSE)
rmse = mse**0.5
print(f'Root Mean Squared Error (RMSE): {rmse}')

Mean Absolute Error (MAE): 63.00420125325521
Mean Squared Error (MSE): 5891.497282636263
Root Mean Squared Error (RMSE): 76.75608954758093


In [38]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror')

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=xg_reg, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=5)

# Fit the GridSearchCV object to the data
grid_search.fit(X_train_scaled, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_estimator.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae}')

print(f"Best Parameters: {best_params}")
print(f"Root Mean Squared Error on Test Set: {rmse}")

Mean Absolute Error (MAE): 60.702915218326595
Best Parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200}
Root Mean Squared Error on Test Set: 75.59224805248185


In [46]:
linear_reg = LinearRegression()

# Define the hyperparameter grid to search (In the case of linear regression, there are no hyperparameters to tune)
param_grid = {}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=linear_reg, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=5)

# Fit the GridSearchCV object to the data
grid_search.fit(X_train_scaled, y_train)

# Get the best estimator
best_estimator = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_estimator.predict(X_test_scaled)

# Evaluate the model using Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae}')

print(f"Best Estimator: {best_estimator}")
print(f"Mean Squared Error on Test Set: {mse}")

Mean Absolute Error (MAE): 48.45555183998941
Best Estimator: LinearRegression()
Mean Squared Error on Test Set: 3051.5710351568964


In [75]:
##df2=df[df['passer_player_name'] == 'P.Mahomes']
df2=df
df2=df2[columns_to_keep]
X = df2.drop('passing_yards', axis=1)
y = df2['passing_yards']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) 
X_test_scaled = scaler.transform(X_test)

lm = LinearRegression()
lm.fit(X_train_scaled, y_train)

y_pred = lm.predict(X_test_scaled)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae}')

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error (MSE): {mse}')

# Calculate Root Mean Squared Error (RMSE)
rmse = mse**0.5
print(f'Root Mean Squared Error (RMSE): {rmse}')

Mean Absolute Error (MAE): 60.54169251798773
Mean Squared Error (MSE): 5649.624728225071
Root Mean Squared Error (RMSE): 75.16398557969815
